In [13]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from multiprocessing import Pool
import time
from tqdm.notebook import tqdm
from utils import get_idle_proportion



""" 
WE NEED FAT NODES FOR RUUNIG THIS.
"""

' \nWE NEED FAT NODES FOR RUUNIG THIS.\n'

In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.precision = 4 # show 4 digits precision
folder_path_slurm_data = Path('./system_analytics_2024/slurm_data')
folder_path_prom_data = Path('./system_analytics_2024/prom_data')
folder_path_saving_results = Path('./results')



In [15]:

# for getting the up-to-date data run the clenaing_sinfo file
# with the latest data from the system.
df = pd.read_parquet(folder_path_slurm_data / 'sinfo_EDA_cleaned.parquet.gzip')
df.sample(n=10)


,node,time,state,node_type,time_30min_interval,time_1hour_interval,time_3hour_interval,time_6hour_interval,time_12hour_interval,time_day_interval
391,tcn425,2024-11-08 19:26:35,allocated,tcn,2024-11-08 19:00:00,2024-11-08 19:00:00,2024-11-08 18:00:00,2024-11-08 18:00:00,2024-11-08 12:00:00,2024-11-08
77,tcn91,2024-11-09 08:35:10,allocated,tcn,2024-11-09 08:30:00,2024-11-09 08:00:00,2024-11-09 06:00:00,2024-11-09 06:00:00,2024-11-09 00:00:00,2024-11-09
62,tcn625,2024-11-05 22:47:48,allocated,tcn,2024-11-05 22:30:00,2024-11-05 22:00:00,2024-11-05 21:00:00,2024-11-05 18:00:00,2024-11-05 12:00:00,2024-11-05
460,tcn502,2024-11-08 15:36:21,allocated,tcn,2024-11-08 15:30:00,2024-11-08 15:00:00,2024-11-08 15:00:00,2024-11-08 12:00:00,2024-11-08 12:00:00,2024-11-08
413,tcn1259,2024-11-07 22:55:46,idle,tcn,2024-11-07 22:30:00,2024-11-07 22:00:00,2024-11-07 21:00:00,2024-11-07 18:00:00,2024-11-07 12:00:00,2024-11-07
19,gcn29,2024-11-10 22:29:45,mixed,gcn,2024-11-10 22:00:00,2024-11-10 22:00:00,2024-11-10 21:00:00,2024-11-10 18:00:00,2024-11-10 12:00:00,2024-11-10
35,fcn45,2024-11-06 05:12:19,idle,fcn,2024-11-06 05:00:00,2024-11-06 05:00:00,2024-11-06 03:00:00,2024-11-06 00:00:00,2024-11-06 00:00:00,2024-11-06
326,tcn481,2024-11-05 13:28:57,allocated,tcn,2024-11-05 13:00:00,2024-11-05 13:00:00,2024-11-05 12:00:00,2024-11-05 12:00:00,2024-11-05 12:00:00,2024-11-05
73,tcn1110,2024-11-08 19:58:47,mixed,tcn,2024-11-08 19:30:00,2024-11-08 19:00:00,2024-11-08 18:00:00,2024-11-08 18:00:00,2024-11-08 12:00:00,2024-11-08
139,tcn903,2024-11-11 09:02:53,idle,tcn,2024-11-11 09:00:00,2024-11-11 09:00:00,2024-11-11 09:00:00,2024-11-11 06:00:00,2024-11-11 00:00:00,2024-11-11


In [16]:
""" 
TEMPORRAY: At the end of the EDA notebook or notebooks save a data frame for ML.
"""
# the time interval that we put all the states in it.
map_time_col = {'time_1hour_interval':pd.Timedelta('1h'),
                'time_30min_interval':pd.Timedelta('30min'),
                'time_3hour_interval':pd.Timedelta('3h')
                }
# pick a time col 
time_col = 'time_3hour_interval'
df_stat = get_idle_proportion(df, time_col)
df_stat.sample(n=10)

,node,time_3hour_interval,idle_duration,all_states_suration,idle_proportion
74291,tcn556,2024-11-07 18:00:00,0 days 00:00:00,0 days 02:59:52,0.0000
94222,tcn809,2024-11-05 06:00:00,0 days 00:00:00,0 days 02:57:33,0.0000
63821,tcn421,2024-11-12 12:00:00,0 days 00:00:00,0 days 02:59:14,0.0000
42317,tcn148,2024-11-04 18:00:00,0 days 00:00:00,0 days 02:59:20,0.0000
51701,tcn268,2024-11-06 06:00:00,0 days 00:00:00,0 days 02:59:25,0.0000
19228,gcn95,2024-11-11 21:00:00,0 days 00:15:02,0 days 02:58:13,0.0844
98014,tcn857,2024-11-08 21:00:00,0 days 02:59:21,0 days 02:59:21,1.0000
87196,tcn72,2024-11-05 15:00:00,0 days 00:00:30,0 days 02:59:11,0.0028
91930,tcn78,2024-11-11 15:00:00,0 days 00:00:00,0 days 02:58:17,0.0000
18679,gcn88,2024-11-05 09:00:00,0 days 00:00:00,0 days 02:57:13,0.0000


In [17]:
# get the last state of the node before the interval
""" 
note the trick here!
"""

df_last = df.groupby(['node', time_col], as_index=False)[['node', time_col, 'state', 'time']].tail(1).copy()

# shift the time without making assumption about the next row
# I am not sure we ned to shift the data
# df_last[time_col] = (df_last[time_col] + map_time_col[time_col]).copy()



df_stat = pd.merge(df_stat, df_last[['node', time_col, 'state', 'time']],
                   how='left', on=['node', time_col]).copy()
df_stat.rename(columns={'time':'time_for_last_state', 'state': 'last_state'}, inplace=True)
df_stat[df_stat['node'].isin(['fcn113', 'fcn114'])].head()


,node,time_3hour_interval,idle_duration,all_states_suration,idle_proportion,last_state,time_for_last_state
1136,fcn113,2024-11-04 15:00:00,0 days 00:56:08,0 days 00:56:08,1.0,idle,2024-11-04 17:59:21
1137,fcn113,2024-11-04 18:00:00,0 days 02:58:50,0 days 02:58:50,1.0,idle,2024-11-04 20:59:41
1138,fcn113,2024-11-04 21:00:00,0 days 02:59:01,0 days 02:59:01,1.0,idle,2024-11-04 23:59:43
1139,fcn113,2024-11-05 00:00:00,0 days 02:58:58,0 days 02:58:58,1.0,idle,2024-11-05 02:59:11
1140,fcn113,2024-11-05 03:00:00,0 days 02:59:33,0 days 02:59:33,1.0,idle,2024-11-05 05:59:45


In [18]:
# get the data from EAR for node waiting time



In [ ]:
""" 
Bring Promethues data set and get the last step or steps of the mesurements ==> 
WE MUST BRING NEWER DATA AS WELL. Is it possible to write a for loops and read the promethues data sets
that we have and do the merging? 
"""
df = pd.read_parquet(folder_path_prom_data / 'metrics_stats_4_sorted.parquet.gzip')
df.head(n=10)

# how to check this df is soreted and does not include duplicates?

,node,rack,timestamp,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds,node_context_switches_total,node_cooling_device_cur_state,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz,node_cpu_frequency_min_hertz,node_cpu_package_throttles_total,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total,node_hwmon_power_average_interval_max_seconds,node_hwmon_power_average_interval_min_seconds,node_hwmon_power_average_interval_seconds,node_hwmon_power_average_watt,node_hwmon_power_is_battery_watt,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total,node_load1,node_load15,node_load5,node_memory_Active_bytes,node_memory_Dirty_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_MemTotal_bytes,node_memory_Percpu_bytes,node_mountstats_nfs_event_vfs_read_page_total,node_mountstats_nfs_event_vfs_read_pages_total,node_mountstats_nfs_event_vfs_setattr_total,node_mountstats_nfs_event_vfs_update_page_total,node_mountstats_nfs_event_vfs_write_page_total,node_mountstats_nfs_event_vfs_write_pages_total,node_mountstats_nfs_read_bytes_total,node_mountstats_nfs_read_pages_total,node_mountstats_nfs_total_read_bytes_total,node_mountstats_nfs_total_write_bytes_total,node_mountstats_nfs_transport_idle_time_seconds_min,node_mountstats_nfs_transport_idle_time_seconds_max,node_mountstats_nfs_transport_idle_time_seconds_mean,node_mountstats_nfs_transport_idle_time_seconds_median,node_mountstats_nfs_transport_receives_total_min,node_mountstats_nfs_transport_receives_total_max,node_mountstats_nfs_transport_receives_total_mean,node_mountstats_nfs_transport_receives_total_median,node_mountstats_nfs_transport_sends_total_min,node_mountstats_nfs_transport_sends_total_max,node_mountstats_nfs_transport_sends_total_mean,node_mountstats_nfs_transport_sends_total_median,node_mountstats_nfs_write_pages_total,node_netstat_Icmp_InErrors,node_netstat_Icmp_InMsgs,node_netstat_Icmp_OutMsgs,node_netstat_Tcp_InErrs,node_netstat_Tcp_InSegs,node_netstat_Tcp_OutSegs,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InDatagrams,node_netstat_Udp_InErrors,node_netstat_Udp_OutDatagrams,node_network_receive_bytes_total_min,node_network_receive_bytes_total_max,node_network_receive_bytes_total_mean,node_network_receive_bytes_total_median,node_network_receive_drop_total_min,node_network_receive_drop_total_max,node_network_receive_drop_total_mean,node_network_recei

In [20]:

# turn the time to pandas time, add the interval tag
df['time'] = pd.to_datetime(df['timestamp'], unit='s')
df[time_col] = ((df['time'] )).dt.floor(freq=map_time_col[time_col])
df.head(n=10)

,node,rack,timestamp,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds,node_context_switches_total,node_cooling_device_cur_state,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz,node_cpu_frequency_min_hertz,node_cpu_package_throttles_total,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total,node_hwmon_power_average_interval_max_seconds,node_hwmon_power_average_interval_min_seconds,node_hwmon_power_average_interval_seconds,node_hwmon_power_average_watt,node_hwmon_power_is_battery_watt,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total,node_load1,node_load15,node_load5,node_memory_Active_bytes,node_memory_Dirty_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_MemTotal_bytes,node_memory_Percpu_bytes,node_mountstats_nfs_event_vfs_read_page_total,node_mountstats_nfs_event_vfs_read_pages_total,node_mountstats_nfs_event_vfs_setattr_total,node_mountstats_nfs_event_vfs_update_page_total,node_mountstats_nfs_event_vfs_write_page_total,node_mountstats_nfs_event_vfs_write_pages_total,node_mountstats_nfs_read_bytes_total,node_mountstats_nfs_read_pages_total,node_mountstats_nfs_total_read_bytes_total,node_mountstats_nfs_total_write_bytes_total,node_mountstats_nfs_transport_idle_time_seconds_min,node_mountstats_nfs_transport_idle_time_seconds_max,node_mountstats_nfs_transport_idle_time_seconds_mean,node_mountstats_nfs_transport_idle_time_seconds_median,node_mountstats_nfs_transport_receives_total_min,node_mountstats_nfs_transport_receives_total_max,node_mountstats_nfs_transport_receives_total_mean,node_mountstats_nfs_transport_receives_total_median,node_mountstats_nfs_transport_sends_total_min,node_mountstats_nfs_transport_sends_total_max,node_mountstats_nfs_transport_sends_total_mean,node_mountstats_nfs_transport_sends_total_median,node_mountstats_nfs_write_pages_total,node_netstat_Icmp_InErrors,node_netstat_Icmp_InMsgs,node_netstat_Icmp_OutMsgs,node_netstat_Tcp_InErrs,node_netstat_Tcp_InSegs,node_netstat_Tcp_OutSegs,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InDatagrams,node_netstat_Udp_InErrors,node_netstat_Udp_OutDatagrams,node_network_receive_bytes_total_min,node_network_receive_bytes_total_max,node_network_receive_bytes_total_mean,node_network_receive_bytes_total_median,node_network_receive_drop_total_min,node_network_receive_drop_total_max,node_network_receive_drop_total_mean,node_network_recei

In [21]:
# get the last time from promethues data base
""" 
note the trick here!
"""
df_last = df.groupby(['node', time_col], as_index=False).tail(1).copy()
df_last.head()


,node,rack,timestamp,node_arp_entries_min,node_arp_entries_max,node_arp_entries_mean,node_arp_entries_median,node_boot_time_seconds,node_context_switches_total,node_cooling_device_cur_state,node_cooling_device_max_state_min,node_cooling_device_max_state_max,node_cooling_device_max_state_mean,node_cooling_device_max_state_median,node_cpu_core_throttles_total,node_cpu_frequency_hertz_min,node_cpu_frequency_hertz_max,node_cpu_frequency_hertz_mean,node_cpu_frequency_hertz_median,node_cpu_frequency_max_hertz,node_cpu_frequency_min_hertz,node_cpu_package_throttles_total,node_disk_io_now_min,node_disk_io_now_max,node_disk_io_now_mean,node_disk_io_now_median,node_disk_read_bytes_total_min,node_disk_read_bytes_total_max,node_disk_read_bytes_total_mean,node_disk_read_bytes_total_median,node_disk_writes_completed_total_min,node_disk_writes_completed_total_max,node_disk_writes_completed_total_mean,node_disk_writes_completed_total_median,node_disk_written_bytes_total_min,node_disk_written_bytes_total_max,node_disk_written_bytes_total_mean,node_disk_written_bytes_total_median,node_filesystem_avail_bytes_min,node_filesystem_avail_bytes_max,node_filesystem_avail_bytes_mean,node_filesystem_avail_bytes_median,node_filesystem_device_error_min,node_filesystem_device_error_max,node_filesystem_device_error_mean,node_filesystem_device_error_median,node_filesystem_files_min,node_filesystem_files_max,node_filesystem_files_mean,node_filesystem_files_median,node_filesystem_files_free_min,node_filesystem_files_free_max,node_filesystem_files_free_mean,node_filesystem_files_free_median,node_filesystem_free_bytes_min,node_filesystem_free_bytes_max,node_filesystem_free_bytes_mean,node_filesystem_free_bytes_median,node_filesystem_size_bytes_min,node_filesystem_size_bytes_max,node_filesystem_size_bytes_mean,node_filesystem_size_bytes_median,node_forks_total,node_hwmon_power_average_interval_max_seconds,node_hwmon_power_average_interval_min_seconds,node_hwmon_power_average_interval_seconds,node_hwmon_power_average_watt,node_hwmon_power_is_battery_watt,node_hwmon_temp_celsius_min,node_hwmon_temp_celsius_max,node_hwmon_temp_celsius_mean,node_hwmon_temp_celsius_median,node_intr_total,node_load1,node_load15,node_load5,node_memory_Active_bytes,node_memory_Dirty_bytes,node_memory_MemAvailable_bytes,node_memory_MemFree_bytes,node_memory_MemTotal_bytes,node_memory_Percpu_bytes,node_mountstats_nfs_event_vfs_read_page_total,node_mountstats_nfs_event_vfs_read_pages_total,node_mountstats_nfs_event_vfs_setattr_total,node_mountstats_nfs_event_vfs_update_page_total,node_mountstats_nfs_event_vfs_write_page_total,node_mountstats_nfs_event_vfs_write_pages_total,node_mountstats_nfs_read_bytes_total,node_mountstats_nfs_read_pages_total,node_mountstats_nfs_total_read_bytes_total,node_mountstats_nfs_total_write_bytes_total,node_mountstats_nfs_transport_idle_time_seconds_min,node_mountstats_nfs_transport_idle_time_seconds_max,node_mountstats_nfs_transport_idle_time_seconds_mean,node_mountstats_nfs_transport_idle_time_seconds_median,node_mountstats_nfs_transport_receives_total_min,node_mountstats_nfs_transport_receives_total_max,node_mountstats_nfs_transport_receives_total_mean,node_mountstats_nfs_transport_receives_total_median,node_mountstats_nfs_transport_sends_total_min,node_mountstats_nfs_transport_sends_total_max,node_mountstats_nfs_transport_sends_total_mean,node_mountstats_nfs_transport_sends_total_median,node_mountstats_nfs_write_pages_total,node_netstat_Icmp_InErrors,node_netstat_Icmp_InMsgs,node_netstat_Icmp_OutMsgs,node_netstat_Tcp_InErrs,node_netstat_Tcp_InSegs,node_netstat_Tcp_OutSegs,node_netstat_Tcp_RetransSegs,node_netstat_Udp_InDatagrams,node_netstat_Udp_InErrors,node_netstat_Udp_OutDatagrams,node_network_receive_bytes_total_min,node_network_receive_bytes_total_max,node_network_receive_bytes_total_mean,node_network_receive_bytes_total_median,node_network_receive_drop_total_min,node_network_receive_drop_total_max,node_network_receive_drop_total_mean,node_network_recei

In [ ]:

df_stat = pd.merge(df_stat, df_last,
                   how='left', on=['node', time_col]).copy()
df_stat.rename(columns={'time':'time_for_last_prom_measurement'}, inplace=True)
df_stat.sample(n=10)
# df_stat[df_stat['node'].isin(['fcn113', 'fcn114'])].head()


